In [41]:
import numpy as np
from scipy.sparse.linalg import svds
from scipy.sparse import coo_matrix, csr_matrix
from implicit.als import AlternatingLeastSquares
from sklearn.decomposition import NMF

In [1]:
import pandas as pd

# Load data
reviews_df = pd.read_csv('../../data/processed/CA_yelp_review.csv')
users_df = pd.read_csv('../../data/processed/CA_yelp_user.csv')
business_df = pd.read_csv('../../data/processed/CA_yelp_business.csv')

In [2]:
users_df

,user_id,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,...,compliment_writer,cool,elite,fans,friends,funny,name,review_count,useful,yelping_since
0,--vbugbG0uZMzODytPRLCg,5.00,0,0,0,0,0,0,0,0,...,0,3,NaN,0,NaN,1,Jennifer,4,7,2015-05-22 00:35:27
1,-1_gsgcmp7PI5VWEQiD3Zg,5.00,0,0,0,0,0,0,0,0,...,0,0,NaN,0,"yFLQkob9zHmieIoq8NPYKw, moecXHkGEaGvUevL-IDtDg...",0,Kristen,3,1,2014-01-19 22:59:29
2,-3Lh4VeOHz6M6wiE1Wj6Vw,5.00,0,0,0,0,0,0,0,0,...,0,0,NaN,0,"lsdrjIiDZnDNO5y1FVMFIA, lre-LmNYfvebpSuhjdUSPg...",0,Maddy,2,0,2014-06-22 21:09:07
3,-3Lh4VeOHz6M6wiE1Wj6Vw,5.00,0,0,0,0,0,0,0,0,...,0,0,NaN,0,"lsdrjIiDZnDNO5y1FVMFIA, lre-LmNYfvebpSuhjdUSPg...",0,Maddy,2,0,2014-06-22 21:09:07
4,-7VZI6A9E8skTgl2NDs19Q,3.33,0,0,0,0,0,0,0,0,...,0,0,NaN,0,jrpY-aiBoUJcmGCEOGXX0A,1,Tony,6,2,2011-09-13 04:41:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348850,zwPXH2G4Z2YwyqSe40IPiQ,2.14,0,0,0,0,0,0,0,0,...,0,2,NaN,0,NaN,1,Jessica,2,19,2011-08-08 13:08:40
348851,zxfu8z5-hSYZ53VJL9pTBA,2.61,0,0,0,0,0,0,2,0,...,1,8,NaN,0,"n4-reEIv27fVCZ709jDw5Q, ui3lk5BkL7Umqe5than-pg...",14,S,52,79,2011-07-11 23:44:25
348852,zxfu8z5-hSYZ53VJL9pTBA,2.61,0,0,0,0,0,0,2,0,...,1,8,NaN,0,"n4-reEIv27fVCZ709jDw5Q, ui3lk5BkL7Umqe5than-pg...",14,S,52,79,2011-07-11 23:44:25
348853,zyApbnUwHafJUvDtiO3Gcw,4.42,2,0,2,1,0,0,2,0,...,0,30,NaN,0,l4pabLd84WrBxt-nHVsZ3g,29,Karen,49,56,2014-07-17 18:05:39


In [3]:
business_df

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,"1616 Chapala St, Ste 2","Row(AcceptsInsurance=None, AgesAllowed=None, A...",Pns2l4eNsfO8kk83dixA6A,"Doctors, Traditional Chinese Medicine, Naturop...",Santa Barbara,NaN,0,34.426679,-119.711197,"Abby Rappoport, LAC, CMQ",93101.0,7,5.0,CA
1,827-833 State St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",noByYNtDLQAra9ccqxdfDw,"Women's Clothing, Accessories, Children's Clot...",Santa Barbara,"Row(Friday='11:0-18:0', Monday='11:0-18:0', Sa...",1,34.420209,-119.700460,H&M,93101.0,24,3.0,CA
2,"131 Anacapa St, Ste C","Row(AcceptsInsurance=None, AgesAllowed=None, A...",IDtLPgUrqorrpqSLdfMhZQ,"Food, Restaurants, Salad, Coffee & Tea, Breakf...",Santa Barbara,"Row(Friday='8:0-14:0', Monday='0:0-0:0', Satur...",1,34.414445,-119.690672,Helena Avenue Bakery,93101.0,389,4.0,CA
3,825 Cacique St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",nUqrF-h9S7myCcvNDecOvw,"Towing, Automotive, Body Shops",Santa Barbara,"Row(Friday='8:0-17:0', Monday='0:0-0:0', Satur...",1,34.419620,-119.677032,Iron Horse Auto Body,93103.0,32,4.5,CA
4,NaN,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",bYjnX_J1bHZob10DoSFkqQ,"Parenting Classes, Maternity Wear, Fashion, Ed...",Santa Barbara,"Row(Friday='10:0-18:0', Monday='0:0-0:0', Satu...",1,34.420334,-119.710749,Tinkle Belle Diaper Service,93101.0,17,5.0,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5198,2012 De La Vina St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",Bo-GGwQNcLVwyKeYkbfAYg,"Event Planning & Services, Wine Tasting Room, ...",Santa Barbara,"Row(Friday='12:0-17:0', Monday='0:0-0:0', Satu...",1,34.429154,-119.717222,Vino Divino,93105.0,19,5.0,CA
5199,900 Embarcadero Del Mar,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",8BUr8GviR2o_b-brO21wwQ,"Restaurants, Burgers, Food, Ice Cream & Frozen...",Isla Vista,NaN,1,34.412966,-119.856886,Kyle's Isla Vista,93117.0,12,2.5,CA
5200,"1805 State St, Ste B","Row(AcceptsInsurance=None, AgesAllowed=None, A...",f7rTfTv6dwO-ovKhz-Rvgw,"Shopping, Eyewear & Opticians, Health & Medica...",Santa Barbara,"Row(Friday='8:0-12:0', Monday='0:0-0:0', Satur...",1,34.428995,-119.712718,"Joanne & Tem Gronquist, OD",93101.0,15,4.5,CA
5201,3534 State St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",uriD7RFuHhLJeDdKaf0nFA,"Restaurants, Pizza, Food",Santa Barbara,"Row(Friday='15:0-21:0', Monday='15:0-21:0', Sa...",0,34.440689,-119.739681,Pizza Guru,93105.0,299,4.0,CA


In [4]:
reviews_df

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,B5XSoSG3SfvQGtKEGQ1tSQ,0,2016-03-30 22:46:33,1,ZKvDG2sBvHVdF5oBNUOpAQ,3.0,This easter instead of going to Lopez Lake we ...,1,wSTuiTk-sKNdcFyprzZAjg
1,gebiRewfieSdtt17PTW6Zg,0,2016-07-25 07:31:06,0,pUycOfUwM8vqX7KjRRhUEA,3.0,Had a party of 6 here for hibachi. Our waitres...,0,59MxRhNVhU9MYndMkz0wtw
2,vC2qm1y3Au5czBtbhc-DNw,0,2013-09-04 03:48:20,0,eCiWBf1CJ0Zdv1uVarEhhw,4.0,"Yes, this is the only sushi place in town. How...",0,OhECKhQEexFypOMY6kypRw
3,bbEXAEFr4RYHLlZ-HFssTA,0,2017-01-02 03:17:34,0,YbMyvlDA2W3Py5lTz8VK-A,5.0,"Great burgers,fries and salad! Burgers have a...",0,4hBhtCSgoxkrFgHa4YAD-w
4,IDtLPgUrqorrpqSLdfMhZQ,0,2016-10-13 22:50:47,0,L0jv8c2FbpWSlfNC6bbUEA,5.0,What a great addition to the Funk Zone! Grab ...,0,bFPdtzu11Oi0f92EAcjqmg
...,...,...,...,...,...,...,...,...,...
348851,fiQxkb1pfPpvrr0oYYYBcA,0,2021-12-06 22:49:57,0,4BknKk03Mxwqp6YoZeu1sA,5.0,"If I could give more than 5 stars, I would! I ...",0,26_8S7BIL6YRw2PcS99Tnw
348852,Kthnb7PjDSd7FsBp4wqoHA,0,2021-12-22 03:58:07,0,IHcg19rAtUsd2LlFMJuO0w,5.0,I have yet to not like a Trader Joe's and this...,0,njEH2bZZtBtKAKfL23JOOw
348853,q2sbzd5Z7Bi8pkmCyr-Umw,0,2021-10-05 18:51:55,0,BnIK6q_3C-6WNXY6M17rRA,5.0,"Mitchel was very professional, timely, and rel...",1,weNjiH8wxEF0xrMtceGQcg
348854,6aUb_f9C7a2oOxaIfRGSdA,1,2019-04-29 18:10:03,0,_USUb1lBWnO4VV58FHL6fw,4.0,I ordered a half rack of Pork Ribs (a la carte...,0,ik0UUieUX38rPg24heeVgw


In [14]:
# Group by 'user_id' and count the number of reviews per user
user_review_counts = reviews_df.groupby('user_id').size().reset_index(name='review_count')

# Sort the users by review count in descending order
user_review_counts = user_review_counts.sort_values(by='review_count', ascending=False)

# Display the top 10 users with the most reviews
top_users = user_review_counts.head(10)
print(top_users)

                       user_id  review_count
88240   ZDCWEctaQHfJQT1sH_rMmA           476
20288   7Na1pUcEv3oF_QTRwZ-2iw           351
484     -B-QEUESGWHPE_889WJaeg           339
127047  o6UJMpHcpLJEvmKLrxLS3w           317
133887  qunxFdBDVl0ordRJEPVNtQ           305
26907   A4bsa7ykYRVCnb4h2vZALw           298
29444   B5s_DCLVrBLrL8U6TEVlwA           292
112954  iJQtNnEcrxdpaD132v26nQ           272
77136   Uk3X2AypU8AqvcYEVf7s6Q           261
55023   LcBZt_bpU5CqwtDJwB3OCg           237


# SVD Proven to not be useful as data is super sparse making all predictions 0

In [15]:
# Create a pivot table with users as rows, businesses as columns, and ratings as values
user_item_matrix = reviews_df.pivot_table(index='user_id', columns='business_id', values='stars').fillna(0)

# Display the matrix
user_item_matrix.head()

business_id,--O3ip9NpXTKD4oBS1pY2A,--onnLZrsCazmcy2P_7fcw,-06ngMH_Ejkm_6HQBYxB7g,-0hxpkIpBh2T0tvdM1mSIw,-1g8Qb6t_mSX_ak1thMmrQ,-1ze-oWDnrGAzvAg56QXUA,-3AooxIkg38UyUdlz5oXdw,-3kTgLg2lUr2PBaokvcnmQ,-6L_z3ftD1iepJb0FfJghw,-6jvfSJGprbfBD2QrS9zQw,...,zppnCAAEn_DstvEc1Uhqvg,zpuFEeAhrNzXPkOlBf5Kog,zsk5qJ179aZvgtz62JCj7A,ztq4yuENNr6nNVb4vyTkpQ,zttPNNUD5QCxm__L39DAmg,zu4p6IZLSVn2Noto-vcwzw,zuB_OGOwQjIr6XWmmD4NQw,zv_3f5U_-7v3UJzpgfhzGQ,zv_X_rMjwJ__sDvjPBVHyA,zxW8zECvT_SqejieMMjb5A
user_id,,,,,,,,,,,,,,,,,,,,,
---zemaUC8WeJeWKqS6p9Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--17Db1K-KujRuN7hY9Z0Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--1M0GJn3SFoXF7R35tszw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--23nA2xXHYNMVJH4bHCug,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--2F5G5LKt3h2cAXJbZptg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Convert the Pandas DataFrame to a scipy sparse matrix
user_item_sparse_matrix = csr_matrix(user_item_matrix.values)

# Decompose the matrix using SVD, k is the number of singular values to keep
U, sigma, Vt = svds(user_item_sparse_matrix, k=100)

# sigma comes out as a 1D array, we convert it into a diagonal matrix
sigma = np.diag(sigma)

In [17]:
# Now you can reconstruct the matrix
reconstructed_matrix = np.dot(np.dot(U, sigma), Vt)

# Convert it back to a DataFrame
approx_user_item_matrix = pd.DataFrame(reconstructed_matrix, index=user_item_matrix.index, columns=user_item_matrix.columns)

# Display the first few rows of the approximated matrix
approx_user_item_matrix.head()

business_id,--O3ip9NpXTKD4oBS1pY2A,--onnLZrsCazmcy2P_7fcw,-06ngMH_Ejkm_6HQBYxB7g,-0hxpkIpBh2T0tvdM1mSIw,-1g8Qb6t_mSX_ak1thMmrQ,-1ze-oWDnrGAzvAg56QXUA,-3AooxIkg38UyUdlz5oXdw,-3kTgLg2lUr2PBaokvcnmQ,-6L_z3ftD1iepJb0FfJghw,-6jvfSJGprbfBD2QrS9zQw,...,zppnCAAEn_DstvEc1Uhqvg,zpuFEeAhrNzXPkOlBf5Kog,zsk5qJ179aZvgtz62JCj7A,ztq4yuENNr6nNVb4vyTkpQ,zttPNNUD5QCxm__L39DAmg,zu4p6IZLSVn2Noto-vcwzw,zuB_OGOwQjIr6XWmmD4NQw,zv_3f5U_-7v3UJzpgfhzGQ,zv_X_rMjwJ__sDvjPBVHyA,zxW8zECvT_SqejieMMjb5A
user_id,,,,,,,,,,,,,,,,,,,,,
---zemaUC8WeJeWKqS6p9Q,-0.005485,-0.000242,3.872656e-04,0.000590,0.001870,-0.000349,0.025436,-2.865921e-05,-0.001239,0.003683,...,0.000323,-0.001342,8.246129e-04,1.312451e-04,-0.001159,0.007568,0.000051,0.001146,0.001130,0.001025
--17Db1K-KujRuN7hY9Z0Q,0.072140,-0.002514,-1.297769e-03,-0.000442,-0.002365,-0.004296,0.024156,-1.973211e-04,0.002480,-0.002496,...,-0.002481,0.003420,-3.382491e-03,-1.790673e-04,0.014332,0.005521,-0.001166,0.006707,-0.001577,0.001879
--1M0GJn3SFoXF7R35tszw,0.000268,-0.000014,2.577146e-07,-0.000001,0.000002,-0.000010,0.000321,1.381993e-06,0.000028,-0.000004,...,0.000010,-0.000027,7.644846e-07,-3.694636e-07,0.000056,0.000007,0.000002,0.000005,0.000003,0.000020
--23nA2xXHYNMVJH4bHCug,0.000173,0.000053,1.408609e-05,0.000007,0.000003,0.000011,0.000692,-2.051431e-07,0.000022,0.000009,...,-0.000002,0.000211,-1.929902e-05,1.436994e-06,0.000010,0.000073,0.000006,0.000013,-0.000010,0.000028
--2F5G5LKt3h2cAXJbZptg,0.007722,-0.001457,-6.284456e-04,-0.000227,-0.002236,0.004319,0.010525,-5.828662e-05,-0.001609,-0.000407,...,-0.000460,0.002507,-7.105709e-04,-3.845105e-05,0.000837,-0.009350,-0.000008,-0.001846,-0.000630,0.002340


# Trying ALS

In [18]:
# Create a pivot table with users as rows and businesses as columns
user_item_matrix = reviews_df.pivot_table(index='user_id', columns='business_id', values='stars').fillna(0)

# Convert to sparse matrix format
sparse_user_item_matrix = csr_matrix(user_item_matrix.values)

# Create a mapping of user and business IDs to indices
user_mapping = {user: idx for idx, user in enumerate(user_item_matrix.index)}
business_mapping = {business: idx for idx, business in enumerate(user_item_matrix.columns)}

In [21]:
# Create ALS model
model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)

# Fit the model
model.fit(sparse_user_item_matrix)

c:\Users\tpngu\anaconda3\envs\food-rec\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 20/20 [00:05<00:00,  3.64it/s]


In [23]:
# Get user and item factors from the model
user_factors = model.user_factors
item_factors = model.item_factors

# Calculate predicted ratings matrix
predicted_ratings = np.dot(user_factors, item_factors.T)

# Convert to DataFrame
predicted_ratings_df = pd.DataFrame(predicted_ratings, 
                                    index=user_item_matrix.index, 
                                    columns=user_item_matrix.columns)

In [24]:
predicted_ratings_df

business_id,--O3ip9NpXTKD4oBS1pY2A,--onnLZrsCazmcy2P_7fcw,-06ngMH_Ejkm_6HQBYxB7g,-0hxpkIpBh2T0tvdM1mSIw,-1g8Qb6t_mSX_ak1thMmrQ,-1ze-oWDnrGAzvAg56QXUA,-3AooxIkg38UyUdlz5oXdw,-3kTgLg2lUr2PBaokvcnmQ,-6L_z3ftD1iepJb0FfJghw,-6jvfSJGprbfBD2QrS9zQw,...,zppnCAAEn_DstvEc1Uhqvg,zpuFEeAhrNzXPkOlBf5Kog,zsk5qJ179aZvgtz62JCj7A,ztq4yuENNr6nNVb4vyTkpQ,zttPNNUD5QCxm__L39DAmg,zu4p6IZLSVn2Noto-vcwzw,zuB_OGOwQjIr6XWmmD4NQw,zv_3f5U_-7v3UJzpgfhzGQ,zv_X_rMjwJ__sDvjPBVHyA,zxW8zECvT_SqejieMMjb5A
user_id,,,,,,,,,,,,,,,,,,,,,
---zemaUC8WeJeWKqS6p9Q,-0.008887,0.000744,0.001650,0.000646,0.002260,0.000060,0.038403,0.000418,-0.003220,0.004940,...,0.001127,-0.000994,-0.004000,3.460158e-04,-0.002978,0.019406,0.000957,0.001422,0.002894,0.002138
--17Db1K-KujRuN7hY9Z0Q,0.031611,0.000004,0.001309,0.000377,0.002087,-0.000323,-0.043706,0.001494,0.001071,-0.003328,...,0.000020,0.023016,0.005605,3.413047e-05,0.004719,0.025316,-0.000193,0.000938,-0.001696,-0.000734
--1M0GJn3SFoXF7R35tszw,0.000591,0.000011,0.000001,-0.000008,0.000026,-0.000019,0.000019,0.000006,-0.000013,0.000026,...,-0.000008,0.000139,-0.000019,-4.324769e-07,0.000190,0.000033,-0.000005,0.000067,0.000026,0.000044
--23nA2xXHYNMVJH4bHCug,0.000269,0.000062,0.000144,0.000042,0.000013,0.000197,0.003174,0.000001,0.000071,0.000178,...,0.000067,0.000910,0.000172,2.752826e-05,0.000108,-0.000114,0.000056,0.000228,-0.000020,0.000002
--2F5G5LKt3h2cAXJbZptg,0.003189,-0.000603,-0.000668,-0.000372,-0.001365,0.000961,0.004138,-0.000308,-0.002057,0.001017,...,-0.000504,-0.002120,-0.007537,4.441368e-04,0.002240,-0.010117,0.000323,-0.001296,0.001162,0.001514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzkJpm2B4PNPHnXg-CMyog,0.012305,0.004667,0.005635,0.001982,0.000669,0.000247,0.064956,-0.001148,-0.014431,0.001468,...,0.005260,0.018739,0.008548,9.740758e-04,0.006325,0.002798,0.000174,-0.000133,-0.002509,0.001611
zzmgn_u1hi1_wdC-lJ452A,0.001337,0.001910,0.000805,0.000385,-0.004273,-0.002517,0.044972,-0.000837,-0.005241,-0.000014,...,-0.000624,0.007763,0.004782,-4.328971e-04,-0.004235,0.016204,0.000312,0.000174,-0.002957,-0.002224
zzpIaMDHg0gvAc7Si6jgiQ,0.000505,0.000023,0.000029,0.000004,-0.000050,-0.000024,0.000292,0.000001,-0.000049,0.000066,...,0.000020,0.000177,0.000006,2.710082e-06,0.000115,0.000114,-0.000006,0.000050,0.000007,0.000048


In [39]:
def predict_rating(user_id, business_id):
    if user_id in predicted_ratings_df.index and business_id in predicted_ratings_df.columns:
        return predicted_ratings_df.loc[user_id, business_id]
    else:
        return None

# Example usage
user_id = 'wSTuiTk-sKNdcFyprzZAjg'
business_id = 'B5XSoSG3SfvQGtKEGQ1tSQ'
predicted_rating = predict_rating(user_id, business_id)
print(f"Predicted Rating for user {user_id} and business {business_id}: {predicted_rating}")

Predicted Rating for user wSTuiTk-sKNdcFyprzZAjg and business B5XSoSG3SfvQGtKEGQ1tSQ: 0.0001272953231818974


SVD and ALS typically do not preserve the exact original ratings due to their nature as low-rank approximation methods. They balance between approximating known values and predicting unknown ones, often leading to some information loss.

In sparse datasets, where the majority of the entries are missing, the algorithms often struggle to find strong signals. As a result:

Noise: The algorithms can be overwhelmed by the noise (missing values), leading to predictions that are closer to the global mean, rather than preserving specific user-item interactions.

Overgeneralization: When the data is very sparse, SVD or ALS tends to overgeneralize, which can result in all predictions clustering around similar values (e.g., ratings close to 0 or close to the average rating).

In [43]:
# Assume df is the original DataFrame with columns 'user_id', 'business_id', 'rating'
# Convert the dataframe into a user-item matrix (pivot table)
user_item_matrix = reviews_df.pivot_table(index='user_id', columns='business_id', values='stars')

# Fill NaNs with 0 (since NMF cannot handle NaNs)
user_item_matrix_filled = user_item_matrix.fillna(0)

# Convert to numpy array (NMF expects a dense matrix)
user_item_matrix_filled = user_item_matrix_filled.values

# Define the number of latent factors
n_factors = 20

# Initialize the NMF model
nmf_model = NMF(n_components=n_factors, init='random', random_state=42, max_iter=200)

# Fit the model on the user-item matrix
W = nmf_model.fit_transform(user_item_matrix_filled)  # User features matrix
H = nmf_model.components_  # Item features matrix

# Reconstruct the user-item matrix using the factorized matrices
user_item_predicted = np.dot(W, H)

# Convert the predicted matrix back to a DataFrame for easier interpretation
user_item_predicted_df = pd.DataFrame(user_item_predicted, 
                                      index=user_item_matrix.index, 
                                      columns=user_item_matrix.columns)

# Print some of the predicted values
print(user_item_predicted_df.head())

business_id             --O3ip9NpXTKD4oBS1pY2A  --onnLZrsCazmcy2P_7fcw  \
user_id                                                                  
---zemaUC8WeJeWKqS6p9Q                0.001965                0.000148   
--17Db1K-KujRuN7hY9Z0Q                0.015793                0.000628   
--1M0GJn3SFoXF7R35tszw                0.000044                0.000003   
--23nA2xXHYNMVJH4bHCug                0.000046                0.000005   
--2F5G5LKt3h2cAXJbZptg                0.005608                0.000337   

business_id             -06ngMH_Ejkm_6HQBYxB7g  -0hxpkIpBh2T0tvdM1mSIw  \
user_id                                                                  
---zemaUC8WeJeWKqS6p9Q            9.872383e-05            3.133596e-05   
--17Db1K-KujRuN7hY9Z0Q            2.196890e-04            1.850773e-04   
--1M0GJn3SFoXF7R35tszw            6.483548e-07            7.463072e-07   
--23nA2xXHYNMVJH4bHCug            2.620852e-06            7.357307e-07   
--2F5G5LKt3h2cAXJbZptg            1.1